In [1]:
import torch
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_val_score

import sys

sys.path.append(".././papers/")

from thdc import cosim, ItemMemory

torch.set_default_device("cpu")

In [3]:
data = pd.read_csv("../datasets/mnist_train.csv")
data = np.array(data)
np.random.shuffle(data)
labels = data[:1000, 0]
data = data[:1000, 1:]
data = torch.from_numpy(data)

In [4]:
vals = torch.randint(0, 2, (256 + 6, 10000), dtype=torch.float)
vals[vals == 0] = -1

board = torch.randint(0, 2, (784, 10000), dtype=torch.float)
board[board == 0] = -1

In [11]:
# def convolution(vs, side=2, weight=20):
#     size = len(vs) - 2 * side
#     width = side * 2 + 1

#     return [sbundle([*vs[i : i + width], weight * vs[i + side]]) for i in range(size)]


# def hdvsc(n, d, side=2, weight=20, iter=5):
#     vs = hdvs(n + iter * side * 2, d)
#     for _ in range(iter):
#         vs = convolution(vs, side, weight)

#     return vs


def convolution(vs, side=2):
    size = vals.size(0) - 2 * side
    width = side * 2 + 1
    nvs = torch.zeros((size, vals.size(1)))

    for i in range(size):
        print(size, i, [i + j for j in range(width)])
        nvs[i] = torch.sum(
            torch.index_select(vs, 0, torch.tensor([i + j for j in range(width)])),
            dim=0,
        )
    return nvs


def conv(n, d, side=2, iter=5):
    vs = torch.randint(0, 2, (n + iter * side * 2, d), dtype=torch.float)
    vs[vs == 0] = -1
    for _ in range(iter):
        vs = convolution(vs, side)

    return vs

In [12]:
# side, weight = 2, 5

# size = vals.size(0) - 2 * side
# width = side * 2 + 1
# nvs = torch.zeros((size, vals.size(1)))

# for i in range(256):
#     nvs[i] = torch.sum(
#         torch.index_select(
#             vals,
#             0,
#             torch.tensor([i, i + 1, i + 2, i + 3, i + 4, i + 5, i + 6]),
#         ),
#         dim=0,
#     )

# print(vals.shape)
# print(nvs.shape)
nvsc = conv(256, 10000, 2)

258 [0, 1, 2, 3, 4]
258 [1, 2, 3, 4, 5]
258 [2, 3, 4, 5, 6]
258 [3, 4, 5, 6, 7]
258 [4, 5, 6, 7, 8]
258 [5, 6, 7, 8, 9]
258 [6, 7, 8, 9, 10]
258 [7, 8, 9, 10, 11]
258 [8, 9, 10, 11, 12]
258 [9, 10, 11, 12, 13]
258 [10, 11, 12, 13, 14]
258 [11, 12, 13, 14, 15]
258 [12, 13, 14, 15, 16]
258 [13, 14, 15, 16, 17]
258 [14, 15, 16, 17, 18]
258 [15, 16, 17, 18, 19]
258 [16, 17, 18, 19, 20]
258 [17, 18, 19, 20, 21]
258 [18, 19, 20, 21, 22]
258 [19, 20, 21, 22, 23]
258 [20, 21, 22, 23, 24]
258 [21, 22, 23, 24, 25]
258 [22, 23, 24, 25, 26]
258 [23, 24, 25, 26, 27]
258 [24, 25, 26, 27, 28]
258 [25, 26, 27, 28, 29]
258 [26, 27, 28, 29, 30]
258 [27, 28, 29, 30, 31]
258 [28, 29, 30, 31, 32]
258 [29, 30, 31, 32, 33]
258 [30, 31, 32, 33, 34]
258 [31, 32, 33, 34, 35]
258 [32, 33, 34, 35, 36]
258 [33, 34, 35, 36, 37]
258 [34, 35, 36, 37, 38]
258 [35, 36, 37, 38, 39]
258 [36, 37, 38, 39, 40]
258 [37, 38, 39, 40, 41]
258 [38, 39, 40, 41, 42]
258 [39, 40, 41, 42, 43]
258 [40, 41, 42, 43, 44]
258 [41, 42, 43

IndexError: index out of range in self

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.interpolate import make_interp_spline, BSpline
from scipy.ndimage import gaussian_filter1d


def similarity_heatmap(xs, targets, round=2, xlabel=None):
    norm = mpl.colors.Normalize(vmin=0, vmax=1.0)

    plt.figure(figsize=(12, 3))
    for n in range(len(targets)):
        sims = []

        for x in xs:
            sims.append(cosim(targets[n], x))

        plt.subplot(len(targets) * 2, 1, n + 1)
        plt.yticks([]), plt.xticks([])
        plt.pcolormesh([sims] * 1, cmap="plasma", norm=norm)
        if round >= 0:
            for i in range(len(xs)):
                plt.text(
                    i + 0.5,
                    0 + 0.5,
                    f"%.{round}f" % sims[i],
                    ha="center",
                    va="center",
                    rotation="vertical",
                    **{"fontname": "JetBrains Mono", "fontsize": 12},
                )
        if xlabel:
            plt.xlabel(xlabel, **{"fontname": "JetBrains Mono", "fontsize": 12})
    plt.show()


nvsc = conv(256, 10000, 4)

similarity_heatmap(nvsc[:50], [nvsc[5]], round=4)

In [ ]:
torch.sum(torch.index_select(vals, 0, torch.tensor([0, 1, 2, 3])), dim=0).shape

In [ ]:
def encode(image):
    values = image[image != 0]
    indices = (image > 0).nonzero().reshape((values.shape[0],))

    vs = vals[values]
    bs = board[indices]

    base = torch.stack((vs, bs), dim=1)
    bin = torch.prod(base, 1)
    G = torch.sum(bin, 0)

    return G

In [ ]:
import random


class MnistClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, step):
        self.memory = ItemMemory()
        self.step = step

    def fit(self, X, y):
        classes = {label: [] for label in set(y)}

        for i in range(len(X)):
            classes[y[i]].append(encode(X[i]))

        for key, value in classes.items():
            for i in range(0, len(value), self.step):
                H = torch.sum(torch.stack(value[i : i + self.step], dim=1), dim=1)
                self.memory.add_vector(key, H)

        return self

    def predict(self, X):
        p = []

        for query in X:
            p.append(self.memory.cleanup(encode(query))[0])
            # p.append(random.randint(0, 10))

        return p

In [ ]:
def main():
    clf = MnistClassifier(step=10000)
    scores = cross_val_score(clf, data, labels, cv=3, n_jobs=1, verbose=4)
    print(scores.mean())


main()